## Imports

In [6]:
# pip install selenium webdriver-manager

# o objetivo deste notebook é olhar para os links que foram scrape e tentar extrair as peças do procedimento

# agora vou primeiro passar isto tudo para um repositório no git lol

import pandas as pd
import polars as pl
import numpy as np

import os

from copy import deepcopy

from utilities.funcoes_scrape import download_pecas_flat
from utilities.schema import contracts_schema, procedimentos_schema
# from utilities.funcoes import preprocessar_contratos

In [ ]:
procedimentos = pl.read_csv("../../data/impic_data/procedimentos.csv",
                            separator=";",
                            schema_overrides=procedimentos_schema(),
                            columns=[
                                "ContractingProcedureAliasID", "Número do Anúncio (Nº/Ano)", "Descrição/designação do procedimento", "Data de Criação", 
                                "URL Peças do Procedimento", "Nº de Lotes"
                                     ],
                            null_values=["NULL"]) \
                            .with_columns(
                                pl.col("Data de Criação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F")
                                            ) \
                            .unique(subset="ContractingProcedureAliasID")


contracts = pl.read_csv("../../data/impic_data/contratos.csv", separator=";", schema_overrides=contracts_schema(),
    columns=['N.º Procedimento (ID BASE)', 'N.º Contrato', 'Data da decisão adjudicação', 
             'Data Celebração', "Contratação Excluída", "Tipo de procedimento", "Ajuste Direto Simplificado"], 
    null_values=["NULL"]) \
    .with_columns(
        pl.col("Data da decisão adjudicação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
        pl.col('Data Celebração').replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
    ) \
    .filter(pl.col("Contratação Excluída") == False) \
    .filter(pl.col("Ajuste Direto Simplificado") == False) \
    .unique(subset=['N.º Procedimento (ID BASE)', 'N.º Contrato'])


contratos_niclas = pl.read_parquet("../../data/contracts_2009_2024.parquet",
                                   columns=[
                                       'contract_id', 'contract_type', 'documents', 'execution_location'
                                   ]) \
                                   .unique(subset = 'contract_id') \
                                   .with_columns(pl.col("contract_id").cast(pl.Utf8))

In [8]:
contratos_cleaned = pl.read_ipc("../../data/clean_up_data/contracts_cleanedup.arrow")

# preprocess contracts
contratos_prepro = contracts.join(contratos_cleaned,
                                  how="anti", 
                                  on = ["N.º Procedimento (ID BASE)", "N.º Contrato"], 
                                  coalesce=True)

In [9]:
# contratos + contratos_niclas
contratos_merged = contratos_prepro.join(contratos_niclas, how='left', left_on='N.º Contrato', right_on='contract_id', coalesce=True)

# só procedimentos com url
procedimentos_com_url = deepcopy(procedimentos.filter(pl.col("URL Peças do Procedimento").is_not_null()))

# todas as colunas que preciso
contratos_merged_final = contratos_merged.join(procedimentos_com_url, how='left', left_on='N.º Procedimento (ID BASE)', right_on='ContractingProcedureAliasID', coalesce=True)

In [15]:
contratos_merged_final = contratos_merged_final.with_columns(
    pl.when(pl.col("URL Peças do Procedimento").is_null()).then(pl.lit(1)).otherwise(pl.lit(0)).alias("flag_no_url"),
    pl.when(pl.col("URL Peças do Procedimento").str.contains("donwload")).then(pl.lit(1)).otherwise(pl.lit(0)).alias("flag_download"),
    pl.when(pl.col("URL Peças do Procedimento").str.contains("vortal")).then(pl.lit(1)).otherwise(pl.lit(0)).alias("flag_vortal")
)

In [17]:
contratos_merged_final.filter(pl.col("Tipo de procedimento")=="Concurso público")

N.º Procedimento (ID BASE),N.º Contrato,Tipo de procedimento,Ajuste Direto Simplificado,Contratação Excluída,Data da decisão adjudicação,Data Celebração,contract_type,documents,execution_location,Número do Anúncio (Nº/Ano),Descrição/designação do procedimento,Data de Criação,URL Peças do Procedimento,Tem Critérios Ambientais?,Tem Especificações Técnicas?,Nº de Lotes,flag_no_url,flag_download,flag_vortal
str,str,str,i64,i64,date,date,str,list[struct[2]],list[struct[3]],str,str,date,str,i64,str,i64,i32,i32,i32
"""4479137""","""6997295""","""Concurso público""",0,0,2020-07-22,2020-09-18,"""Aquisição de bens móveis""","[{""ContratoSodarcaassinadoempresa CG_BASE(23NOV21).pdf"",1412552}]","[{""Portugal"",""Lisboa"",""Lisboa""}]","""3707/2020""","""CP N.º 07/DRL/DA/2020 - Aquis…",2020-04-07,"""https://community.vortal.biz/P…",null,null,9,0,0,1
"""5319991""","""7797066""","""Concurso público""",0,0,2021-08-04,2021-08-11,"""Aquisição de bens móveis""","[{""Contrato expurgado Arcgis.pdf"",1053678}]","[{""Portugal"",""Lisboa"",""Lisboa""}]","""7032/2021""","""Aquisição de atualizações de l…",2021-05-27,"""https://www.anogov.com/op-dgrm…",null,null,1,0,0,0
"""4140333""","""6027302""","""Concurso público""",0,0,2019-11-07,2019-11-07,"""Aquisição de bens móveis""",null,"[{""Portugal"",null,null}]",null,null,null,null,null,null,null,1,0,0
"""4409712""","""6628486""","""Concurso público""",0,0,2020-04-27,2020-05-08,"""Aquisição de serviços""","[{""CP 04-20_Ocultado.pdf"",1098218}]","[{""Portugal"",""Lisboa"",""Lisboa""}]",null,null,null,null,null,null,null,1,0,0
"""5440715""","""8829634""","""Concurso público""",0,0,2021-11-25,2021-11-25,"""Aquisição de bens móveis""",null,"[{""Portugal"",""Setubal"",""Barreiro""}]","""11217/2021""","""1110003/2022 - MEDICAMENTOS DI…",2021-08-31,"""https://community.vortal.biz/P…",null,null,17,0,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""4154232""","""6269302""","""Concurso público""",0,0,2019-11-28,2020-01-27,"""Empreitadas de obras públicas""","[{""CONTRATO.pdf"",1326198}]","[{""Portugal"",""Porto"",""Matosinhos""}]",null,null,null,null,null,null,null,1,0,0
"""3409837""","""5302525""","""Concurso público""",0,0,2018-05-25,2018-05-28,"""Aquisição de serviços""","[{""Contrato Willis.pdf"",1348501}]","[{""Portugal"",""Porto"",""Penafiel""}, {""Portugal"",""Porto"",""Amarante""}]",null,null,null,null,null,null,null,1,0,0
"""3794529""","""5955541""","""Concurso público""",0,0,2019-09-09,2019-10-14,"""Aquisição de bens móveis""","[{""ContratoN.º 147-2019_Fujifilm_EXPURGADO.pdf"",1292613}]","[{""Portugal"",""Castelo Branco"",""Castelo Branco""}, {null,null,""Covilha""}]",null,null,null,null,null,null,null,1,0,0


In [ ]:
# só contratos de 2023 com documentos 
contratos_extract = contratos_merged_final.filter((pl.col("URL Peças do Procedimento").is_not_null()) & (pl.col("Data Celebração").dt.year()==2022)) #& (pl.col("documents").is_not_null()))

In [13]:
contratos_extract.filter(
    (pl.col('URL Peças do Procedimento').str.contains('donwload'))).shape

(101303, 17)

In [14]:
# ver os que já foram extraídos

ids_extraidos = []
for item in os.listdir('../../docs_pecas'):
    ids_extraidos.append(item)

In [16]:
len(ids_extraidos)

26099

In [128]:
# ver os que podemos extrair com download as pecas do procedimento

contratos_pecas_download = contratos_extract.filter(
    (pl.col('URL Peças do Procedimento').str.contains('donwload')) &
    (~pl.col("N.º Procedimento (ID BASE)").cast(pl.Utf8).is_in(ids_extraidos))
).unique(subset='N.º Procedimento (ID BASE)')

In [129]:
contratos_pecas_download_pd = contratos_pecas_download.to_pandas()

In [130]:
contratos_pecas_download_pd.shape

(20441, 17)

In [131]:
to_extract = contratos_pecas_download_pd[:1000]

In [132]:
to_extract.apply(download_pecas_flat, axis=1, out_root="../../docs_pecas")

0      {'ok': True, 'N.º Procedimento (ID BASE)': '63...
1      {'ok': True, 'N.º Procedimento (ID BASE)': '60...
2      {'ok': True, 'N.º Procedimento (ID BASE)': '57...
3      {'ok': True, 'N.º Procedimento (ID BASE)': '59...
4      {'ok': True, 'N.º Procedimento (ID BASE)': '58...
                             ...                        
995    {'ok': True, 'N.º Procedimento (ID BASE)': '60...
996    {'ok': True, 'N.º Procedimento (ID BASE)': '58...
997    {'ok': True, 'N.º Procedimento (ID BASE)': '61...
998    {'ok': True, 'N.º Procedimento (ID BASE)': '60...
999    {'ok': True, 'N.º Procedimento (ID BASE)': '58...
Length: 1000, dtype: object

#### Caso vortal

In [ ]:
contratos_extract_vortal= contratos_extract[contratos_extract['URL Peças do Procedimento'].str.contains('vortal', case=False, na=False, regex=True)]
contratos_pecas_vortal = contratos_extract_vortal.drop_duplicates(subset='N.º Procedimento (ID BASE)')